# Część 1: Przygotowanie Danych i Inżynieria Cech
Ta część jest odpowiedzialna za wczytanie, oczyszczenie i przetworzenie danych w celu przygotowania ich do treningu modelu. Zapiszemy również wszystkie niezbędne artefakty (mapy, transformatory) na dysku.

In [1]:
import pandas as pd
import numpy as np
import os
import re
import joblib
import gc
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

# Definicja stałych globalnych
ARTIFACTS_DIR = 'artifacts'
PROCESSED_DATA_DIR = 'processed_data'
MAX_TEXT_FEATURES = 10000
RANDOM_STATE = 42

# Tworzenie katalogów, jeśli nie istnieją
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"Katalogi '{ARTIFACTS_DIR}' i '{PROCESSED_DATA_DIR}' są gotowe.")

Katalogi 'artifacts' i 'processed_data' są gotowe.


## Wczytywanie Danych
W tej komórce wczytujemy wszystkie niezbędne pliki CSV: dane o ofertach, definicję hierarchii lokalizacji oraz mapę do standaryzacji nazw.

In [2]:
# KOMÓRKA 2 (Kod) - POPRAWIONA, Z WŁAŚCIWYM SEPARATOREM

def load_location_data(location_path='lokalizacja.csv'):
    """Wczytuje dane o hierarchii lokalizacji."""
    print(f"Wczytywanie pliku: {location_path}")
    # KLUCZOWA ZMIANA: Wracamy do przecinka, bo dane są nim rozdzielone.
    # Dodajemy nazwy kolumn, bo plik ich nie ma.
    df_loc = pd.read_csv(
        location_path,
        na_values=['\\N', 'NULL'],
        sep=',',
        header=None,
        names=['id', 'parent_id', 'name', 'type', 'full_name'] # Zgodnie z promptem + dodatkowe
    )
    return df_loc

def load_offers_data(offers_path='saleflats_2024_2025.csv'):
    """Wczytuje dane o ofertach, dynamicznie znajdując ostatnią kolumnę."""
    print(f"Wczytywanie pliku: {offers_path}")
    column_names = ['title', 'description', 'area', 'price', 'locationPath']
    
    try:
        # KLUCZOWA ZMIANA: Wracamy do przecinka
        first_row = pd.read_csv(offers_path, header=None, sep=',', nrows=1, on_bad_lines='skip')
        last_col_index = first_row.shape[1] - 1
        print(f"Wykryto {last_col_index + 1} kolumn. Ostatnia kolumna ma indeks: {last_col_index}")
        
        usecols = [3, 4, 5, 6, last_col_index]
        
        df_offers = pd.read_csv(offers_path, header=None, sep=',', usecols=usecols, on_bad_lines='skip')
        df_offers.columns = column_names
        
    except Exception as e:
        print(f"Wystąpił błąd: {e}. Upewnij się, że plik '{offers_path}' ma co najmniej 7 kolumn i używa przecinka jako separatora.")
        df_offers = pd.DataFrame(columns=column_names)
        
    return df_offers

# Wywołanie funkcji
df_loc = load_location_data()
df_offers = load_offers_data()

# Wyświetlenie informacji o wczytanych danych
print("\n--- Informacje o danych lokalizacyjnych (df_loc) ---")
df_loc.info()
print("\n--- Nagłówek danych lokalizacyjnych ---")
print(df_loc.head())

print("\n--- Informacje o danych ofert (df_offers) ---")
df_offers.info()
print("\n--- Nagłówek danych ofert ---")
print(df_offers.head())

Wczytywanie pliku: lokalizacja.csv
Wczytywanie pliku: saleflats_2024_2025.csv
Wykryto 53 kolumn. Ostatnia kolumna ma indeks: 52

--- Informacje o danych lokalizacyjnych (df_loc) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398832 entries, 0 to 398831
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         398832 non-null  int64  
 1   parent_id  398816 non-null  float64
 2   name       398832 non-null  object 
 3   type       398832 non-null  object 
 4   full_name  398832 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 15.2+ MB

--- Nagłówek danych lokalizacyjnych ---
   id  parent_id                name         type           full_name
0   1        NaN      Świętokrzyskie  Województwo      Świętokrzyskie
1   2        NaN             Śląskie  Województwo             Śląskie
2   3        NaN           Podlaskie  Województwo           Podlaskie
3   4        NaN            Opolskie 

## Budowa Hierarchii i Standaryzacja
Tworzymy kluczowe mapy (`id_to_name`, `hierarchy_map`) na podstawie danych o lokalizacjach. Standaryzujemy również nazwy w obu ramkach danych, aby zapewnić spójność.

In [3]:
# KOMÓRKA 3 (Kod) - POPRAWIONA

# 1. Stworzenie i zapisanie mapy id -> nazwa
# Upewniamy się, że kolumny są poprawnie nazwane (zgodnie z promptem)
id_to_name = dict(zip(df_loc['id'], df_loc['name']))
joblib.dump(id_to_name, os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
print(f"Mapa 'id_to_name' została stworzona i zapisana. Liczba wpisów: {len(id_to_name)}")

# 2. Stworzenie i zapisanie mapy hierarchii
hierarchy_map = dict(zip(df_loc['id'], df_loc['parent_id']))
joblib.dump(hierarchy_map, os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
print(f"Mapa 'hierarchy_map' została stworzona i zapisana. Liczba wpisów: {len(hierarchy_map)}")

print("\n--- Przykładowe ścieżki (bez zmian, bo to już ID) ---")
print(df_offers[['locationPath']].head())

Mapa 'id_to_name' została stworzona i zapisana. Liczba wpisów: 398832
Mapa 'hierarchy_map' została stworzona i zapisana. Liczba wpisów: 398832

--- Przykładowe ścieżki (bez zmian, bo to już ID) ---
               locationPath
0      3,0,0,352,0,103786,0
1       3,0,0,352,0,99764,0
2       3,0,0,352,0,74375,0
3  3,0,0,352,0,74375,517513
4       3,0,0,352,0,95559,0


## Inżynieria Cech i Definicja Zmiennej Celu
W tej sekcji tworzymy zmienną celu (`target_location_id`) oraz cechy, które posłużą do treningu modelu: połączone cechy tekstowe, cechy numeryczne oraz cechę kategoryczną (miasto).

In [4]:
# KOMÓRKA 4 (Kod) - ZAKTUALIZOWANA WERSJA

# --- Cechy Numeryczne (z nową cechą) ---
df_offers['area'] = pd.to_numeric(df_offers['area'], errors='coerce')
df_offers['price'] = pd.to_numeric(df_offers['price'], errors='coerce')
# Unikamy dzielenia przez zero lub NaN
df_offers.dropna(subset=['area', 'price'], inplace=True)
df_offers = df_offers[df_offers['area'] > 0]
# Obliczamy nową cechę
df_offers['price_per_meter'] = df_offers['price'] / df_offers['area']

# --- Nowa definicja celów dla modelu Multi-Head ---
# Parsujemy ścieżkę na oddzielne kolumny, aby łatwiej wyciągnąć cele
path_cols = ['woj_id', 'pow_id', 'gmi_id', 'city_id', 'district_id', 'subdistrict_id', 'street_id']
path_df = df_offers['locationPath'].str.split(',', expand=True)
path_df = path_df.iloc[:, :len(path_cols)]
path_df.columns = path_cols[:path_df.shape[1]]
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# Łączymy z powrotem z główną ramką danych
df_offers = pd.concat([df_offers.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)

# Definiujemy cele
# Cel "dzielnica": jeśli jest pod-dzielnica, bierzemy ją, jeśli nie, bierzemy dzielnicę. Jeśli nie ma żadnej, dajemy 0.
df_offers['target_district_id'] = np.where(df_offers['subdistrict_id'] != 0, df_offers['subdistrict_id'], df_offers['district_id'])
# Cel "ulica": bierzemy ID ulicy, jeśli nie ma, dajemy 0.
df_offers['target_street_id'] = df_offers['street_id']

# --- Finalne czyszczenie i tworzenie df_processed ---
df_offers.dropna(subset=['city_id'], inplace=True)
df_offers = df_offers[df_offers['city_id'] != 0]

df_processed = df_offers[[
    'title', 'description', 'area', 'price', 'price_per_meter', 
    'city_id', 'target_district_id', 'target_street_id'
]].copy()
df_processed['text_features'] = df_processed['title'].fillna('') + " " + df_processed['description'].fillna('')
df_processed.drop(columns=['title', 'description'], inplace=True)
df_processed.dropna(inplace=True)

print("\n--- Informacje o finalnym, przetworzonym DataFrame (df_processed) ---")
df_processed.info()
print("\n--- Nagłówek df_processed ---")
print(df_processed.head())


--- Informacje o finalnym, przetworzonym DataFrame (df_processed) ---
<class 'pandas.core.frame.DataFrame'>
Index: 1112222 entries, 0 to 1122941
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   area                1112222 non-null  float64
 1   price               1112222 non-null  float64
 2   price_per_meter     1112222 non-null  float64
 3   city_id             1112222 non-null  int32  
 4   target_district_id  1112222 non-null  int32  
 5   target_street_id    1112222 non-null  int32  
 6   text_features       1112222 non-null  object 
dtypes: float64(3), int32(3), object(1)
memory usage: 55.2+ MB

--- Nagłówek df_processed ---
    area     price  price_per_meter  city_id  target_district_id  \
0  73.00  766500.0     10500.000000      352              103786   
1  64.80  540000.0      8333.333333      352               99764   
2  51.00  540000.0     10588.235294      352               74375 

## Transformacja Cech i Podział na Zbiory
Dzielimy dane na zbiory treningowy i walidacyjny. Następnie dopasowujemy transformatory (imputer, scaler, vectorizer) na zbiorze treningowym i zapisujemy je, aby móc ich użyć w przyszłości.

In [5]:
# KOMÓRKA 5 (Kod) - ZAKTUALIZOWANA WERSJA

# --- KLUCZOWA ZMIANA: Filtracja klas z jednym członkiem ---
print("Sprawdzanie liczności klas dla stratyfikacji...")
city_counts = df_processed['city_id'].value_counts()
valid_cities = city_counts[city_counts > 1].index
df_filtered = df_processed[df_processed['city_id'].isin(valid_cities)].copy()
print(f"Usunięto {len(df_processed) - len(df_filtered)} wierszy należących do miast z tylko jednym ogłoszeniem.")

# Podział na zbiory
X = df_filtered.drop(columns=['target_district_id', 'target_street_id'])
y = df_filtered[['target_district_id', 'target_street_id']]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=X['city_id']
)

print(f"Podział danych: {len(X_train)} próbek treningowych, {len(X_val)} próbek walidacyjnych.")

# --- Przetwarzanie Cech Numerycznych (z nową cechą) ---
numeric_features = ['area', 'price', 'price_per_meter']
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

X_train_num_imputed = imputer.fit_transform(X_train[numeric_features])
X_train_num = scaler.fit_transform(X_train_num_imputed)

joblib.dump(imputer, os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))
print("Imputer i Scaler zostały dopasowane i zapisane.")

X_val_num_imputed = imputer.transform(X_val[numeric_features])
X_val_num = scaler.transform(X_val_num_imputed)

# --- Przetwarzanie Cech Tekstowych ---
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES, ngram_range=(1, 2))
X_train_text = vectorizer.fit_transform(X_train['text_features'])

joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))
print("TfidfVectorizer został dopasowany i zapisany.")

X_val_text = vectorizer.transform(X_val['text_features'])

# --- Przygotowanie pozostałych cech ---
X_train_city = X_train['city_id'].values
X_val_city = X_val['city_id'].values

# --- Przygotowanie oddzielnych celów ---
y_district_train = y_train['target_district_id'].values
y_street_train = y_train['target_street_id'].values
y_district_val = y_val['target_district_id'].values
y_street_val = y_val['target_street_id'].values

Sprawdzanie liczności klas dla stratyfikacji...
Usunięto 2757 wierszy należących do miast z tylko jednym ogłoszeniem.
Podział danych: 887572 próbek treningowych, 221893 próbek walidacyjnych.
Imputer i Scaler zostały dopasowane i zapisane.
TfidfVectorizer został dopasowany i zapisany.


## Zapisanie Wyników do Plików
Wszystkie przetworzone dane oraz kluczowe ramki danych są zapisywane na dysku. Umożliwi to ich wczytanie w drugiej części notebooka bez potrzeby ponownego przetwarzania.

In [7]:
# KOMÓRKA 6 (Kod) - ZAKTUALIZOWANA WERSJA

# Zapis macierzy TF-IDF
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'), X_train_text)
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'), X_val_text)

# Zapis macierzy numerycznych
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'), X_train_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'), X_val_num)

# Zapis ID miast
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'), X_train_city)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'), X_val_city)

# Zapis ODDZIELNYCH zmiennych docelowych
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_district_train.npy'), y_district_train)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_street_train.npy'), y_street_train)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_district_val.npy'), y_district_val)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_street_val.npy'), y_street_val)

print("Wszystkie przetworzone zbiory danych zostały zapisane.")

# Zwolnienie pamięci
del df_loc, df_offers, df_processed, X, y, X_train, X_val, y_train, y_val
del X_train_text, X_val_text, X_train_num, X_val_num, X_train_city, X_val_city
gc.collect()

print("\n" + "="*80)
print("CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM.")
print("Wszystkie dane i artefakty zostały zapisane w katalogach 'processed_data' i 'artifacts'.")
print("TERAZ NALEŻY ZRESTARTOWAĆ KERNEL PRZED URUCHOMIENIEM CZĘŚCI 2.")
print("="*80)

Wszystkie przetworzone zbiory danych zostały zapisane.

CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM.
Wszystkie dane i artefakty zostały zapisane w katalogach 'processed_data' i 'artifacts'.
TERAZ NALEŻY ZRESTARTOWAĆ KERNEL PRZED URUCHOMIENIEM CZĘŚCI 2.


# Część 2: Budowa, Trening i Inferencia Modelu
Witaj w nowej sesji kernela! Ta część wczyta przetworzone dane i artefakty, a następnie zdefiniuje, wytrenuje i wykorzysta model sieci neuronowej do predykcji lokalizacji z uwzględnieniem spójności hierarchicznej.

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import gc
import scipy.sparse
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Ponowna definicja stałych
ARTIFACTS_DIR = 'artifacts'
PROCESSED_DATA_DIR = 'processed_data'
RANDOM_STATE = 42
MAX_TEXT_FEATURES = 10000 
MODEL_PATH = os.path.join(ARTIFACTS_DIR, 'best_location_model.keras')

## Wczytywanie Przetworzonych Danych i Artefaktów
Wczytujemy wszystkie niezbędne pliki wygenerowane w Części 1.

In [6]:
# KOMÓRKA 8 (Kod) - POPRAWIONA I UZUPEŁNIONA

# Wczytanie artefaktów
print("Wczytywanie artefaktów...")
hierarchy_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
id_to_name = joblib.load(os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
scaler = joblib.load(os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))
imputer = joblib.load(os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))
vectorizer = joblib.load(os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))
print("Artefakty wczytane.")

# Wczytanie przetworzonych danych
print("Wczytywanie przetworzonych danych...")
X_train_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'))
X_val_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'))
X_train_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'))
X_val_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'))
X_train_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'))
X_val_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'))

# --- KLUCZOWA ZMIANA: Wczytanie oddzielnych celów dla modelu Multi-Head ---
y_district_train = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_district_train.npy'))
y_street_train = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_street_train.npy'))
y_district_val = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_district_val.npy'))
y_street_val = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_street_val.npy'))
# --- KONIEC ZMIANY ---

print("Dane wczytane.")

# Wyświetlanie kształtów
print("\n--- Kształty wczytanych danych ---")
print(f"X_train_text: {X_train_text.shape}")
print(f"X_train_num:  {X_train_num.shape}")
print(f"X_train_city: {X_train_city.shape}")
print(f"y_district_train: {y_district_train.shape}")
print(f"y_street_train:   {y_street_train.shape}")

Wczytywanie artefaktów...
Artefakty wczytane.
Wczytywanie przetworzonych danych...
Dane wczytane.

--- Kształty wczytanych danych ---
X_train_text: (887572, 10000)
X_train_num:  (887572, 3)
X_train_city: (887572,)
y_district_train: (887572,)
y_street_train:   (887572,)


In [10]:
# KOMÓRKA 8a (Kod) - OSTATECZNA, POPRAWNA WERSJA

print("Definiowanie hierarchicznej funkcji straty...")

# Wczytanie pliku lokalizacja.csv
print("Wczytywanie pliku lokalizacja.csv na potrzeby funkcji straty...")
df_loc = pd.read_csv(
    'lokalizacja.csv',
    na_values=['\\N', 'NULL'],
    sep=',',
    header=None,
    names=['id', 'parent_id', 'name', 'type', 'full_name']
)

# Przygotowanie map hierarchii
id_to_parent = dict(zip(df_loc['id'], df_loc['parent_id']))
id_to_type = dict(zip(df_loc['id'], df_loc['type']))

def get_city_for_id(loc_id, id_to_parent_map, id_to_type_map):
    current_id = loc_id
    for _ in range(5):
        if pd.isna(current_id) or current_id == 0:
            return 0
        if id_to_type_map.get(current_id) == 'CITY':
            return int(current_id)
        current_id = id_to_parent_map.get(current_id)
    return 0

all_ids = list(id_to_parent.keys())
id_to_city_map = {loc_id: get_city_for_id(loc_id, id_to_parent, id_to_type) for loc_id in all_ids}
joblib.dump(id_to_city_map, os.path.join(ARTIFACTS_DIR, 'id_to_city_map.joblib'))
print("Mapa 'id_to_city' stworzona i zapisana.")


# --- KLUCZOWA ZMIANA: Dodajemy dekorator i metody get_config/from_config ---
@tf.keras.saving.register_keras_serializable()
class HierarchicalLoss(tf.keras.losses.Loss):
    def __init__(self, id_to_parent_map, id_to_city_map, penalty_config, **kwargs):
        super().__init__(**kwargs)
        self.base_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        
        # Przechowujemy oryginalne mapy i config, aby je zapisać
        self.id_to_parent_map = id_to_parent_map
        self.id_to_city_map = id_to_city_map
        self.penalty_config = penalty_config
        
        self.penalty_wrong_parent = penalty_config.get('wrong_parent', 2.0)
        self.penalty_wrong_city = penalty_config.get('wrong_city', 5.0)

        keys = list(id_to_parent_map.keys())
        values = [int(v) if pd.notna(v) else 0 for v in id_to_parent_map.values()]
        self.parent_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.int64, value_dtype=tf.int64),
            default_value=0
        )
        
        keys = list(id_to_city_map.keys())
        values = [int(v) for v in id_to_city_map.values()]
        self.city_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.int64, value_dtype=tf.int64),
            default_value=0
        )

    def call(self, y_true, y_pred):
        y_true_tensor = tf.cast(y_true, dtype=tf.int64)
        y_pred_ids = tf.cast(tf.argmax(y_pred, axis=-1), dtype=tf.int64)

        base_loss = self.base_loss_fn(y_true_tensor, y_pred)
        
        true_parents = self.parent_table.lookup(y_true_tensor)
        pred_parents = self.parent_table.lookup(y_pred_ids)
        true_cities = self.city_table.lookup(y_true_tensor)
        pred_cities = self.city_table.lookup(y_pred_ids)
        
        penalties = tf.ones_like(base_loss) * self.penalty_wrong_city
        penalties = tf.where(tf.equal(true_cities, pred_cities), tf.ones_like(base_loss) * self.penalty_wrong_parent, penalties)
        penalties = tf.where(tf.equal(true_parents, pred_parents), tf.ones_like(base_loss) * 1.2, penalties)
        penalties = tf.where(tf.equal(y_true_tensor, y_pred_ids), tf.ones_like(base_loss), penalties)

        return tf.reduce_mean(base_loss * penalties)

    def get_config(self):
        # Ta metoda mówi Kerasowi, co zapisać w pliku modelu
        base_config = super().get_config()
        config = {
            "id_to_parent_map": self.id_to_parent_map,
            "id_to_city_map": self.id_to_city_map,
            "penalty_config": self.penalty_config,
        }
        return {**base_config, **config}

    @classmethod
    def from_config(cls, config):
        # Ta metoda mówi Kerasowi, jak odtworzyć klasę z zapisanego pliku
        return cls(**config)
# --- KONIEC ZMIANY ---

print("Hierarchiczna funkcja straty zdefiniowana i gotowa do serializacji.")

Definiowanie hierarchicznej funkcji straty...
Wczytywanie pliku lokalizacja.csv na potrzeby funkcji straty...
Mapa 'id_to_city' stworzona i zapisana.
Hierarchiczna funkcja straty zdefiniowana.


## Definicja Architektury Modelu (Multi-Input, Single-Output)
Budujemy model Keras z trzema oddzielnymi wejściami (tekst, cechy numeryczne, miasto), które są łączone i przetwarzane przez wspólne warstwy gęste.

In [11]:
# KOMÓRKA 9 (Kod) - NOWA ARCHITEKTURA MULTI-HEAD

# --- Mapowanie ID na ciągłe indeksy (tak jak w v6) ---
# Robimy to dla każdego celu oddzielnie
unique_districts = np.unique(np.concatenate([y_district_train, y_district_val]))
district_map = {val: i for i, val in enumerate(unique_districts)}
y_district_train = np.array([district_map.get(val, 0) for val in y_district_train])
y_district_val = np.array([district_map.get(val, 0) for val in y_district_val])

unique_streets = np.unique(np.concatenate([y_street_train, y_street_val]))
street_map = {val: i for i, val in enumerate(unique_streets)}
y_street_train = np.array([street_map.get(val, 0) for val in y_street_train])
y_street_val = np.array([street_map.get(val, 0) for val in y_street_val])

# Definicja parametrów
NUM_DISTRICTS = len(unique_districts)
NUM_STREETS = len(unique_streets)
NUM_CITIES = int(np.concatenate([X_train_city, X_val_city]).max() + 1)
NUM_FEATURES = X_train_num.shape[1]

# --- Definicja modelu ---
input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True)
input_num = Input(shape=(NUM_FEATURES,), name='num_input')
input_city = Input(shape=(1,), name='city_input')

# Wspólna część modelu (ciało)
text_branch = Dense(128, activation='relu')(input_text)
text_branch = Dropout(0.3)(text_branch)

num_branch = Dense(64, activation='relu')(input_num)
num_branch = Dense(32, activation='relu')(num_branch)

city_branch = Embedding(input_dim=NUM_CITIES, output_dim=50, name='city_embedding')(input_city)
city_branch = Flatten()(city_branch)

combined = Concatenate()([text_branch, num_branch, city_branch])

z = Dense(512, activation='relu')(combined)
z = Dropout(0.5)(z)
z = Dense(256, activation='relu')(z)
z = Dropout(0.5)(z)

# Oddzielne głowice wyjściowe
output_district = Dense(NUM_DISTRICTS, activation='softmax', name='district_output')(z)
output_street = Dense(NUM_STREETS, activation='softmax', name='street_output')(z)

model = Model(inputs=[input_text, input_num, input_city], outputs=[output_district, output_street])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 10000)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 3)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 128)               │       1,280,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 64)                │             256 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_embedding (Embedding)    │ (None, 1, 50)             │       3,150,200 │ city_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 128)               │               0 │ dense_5[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 32)                │           2,080 │ dense_6[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 50)                │               0 │ city_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 210)               │               0 │ dropout_3[0][0],           │
│                               │                           │                 │ dense_7[0][0],             │
│                               │                           │                 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 512)               │         108,032 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_4 (Dropout)           │ (None, 512)               │               0 │ dense_8[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 256)               │         131,328 │ dropout_4[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_5 (Dropout)           │ (None, 256)               │               0 │ dense_9[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ district_output (Dense)       │ (None, 2204)              │         566,428 │ dropout_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ street_output (Dense)         │ (None, 33604)             │       8,636,228 │ dropout_5[0][0]            │
└───────────────────────────────┴───────────────────────────┴───────────────

 Total params: 13,874,680 (52.93 MB)

 Trainable params: 13,874,680 (52.93 MB)

 Non-trainable params: 0 (0.00 B)

## Trening Modelu
Uruchamiamy proces treningu z wykorzystaniem `EarlyStopping` w celu uniknięcia przeuczenia oraz `ModelCheckpoint` do zapisania najlepszej wersji modelu.

In [13]:
# KOMÓRKA 10 (Kod) - POPRAWIONA I UZUPEŁNIONA

# Przygotowanie map hierarchii dla funkcji straty (oryginalne ID)
id_to_parent_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
id_to_city_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'id_to_city_map.joblib'))

# Stworzenie instancji naszych funkcji straty
penalty_config = {'wrong_parent': 2.0, 'wrong_city': 5.0}
district_loss = HierarchicalLoss(id_to_parent_map, id_to_city_map, penalty_config, name='district_hierarchical_loss')
street_loss = HierarchicalLoss(id_to_parent_map, id_to_city_map, penalty_config, name='street_hierarchical_loss')

# Kompilacja modelu z dwiema funkcjami strat i wagami
model.compile(
    optimizer='adam',
    loss={
        'district_output': district_loss,
        'street_output': street_loss
    },
    loss_weights={
        'district_output': 1.2, 
        'street_output': 1.0
    },
    metrics={'district_output': 'accuracy', 'street_output': 'accuracy'}
)

# --- KLUCZOWA ZMIANA: Dodanie definicji callbacków ---
# Zapisuje najlepszy model na podstawie błędu walidacyjnego
model_checkpoint = ModelCheckpoint(
    MODEL_PATH, 
    monitor='val_loss', 
    save_best_only=True,
    verbose=1
)

# Przerywa trening, jeśli błąd walidacyjny nie poprawia się przez 3 epoki
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3,
    restore_best_weights=True,
    verbose=1
)
# --- KONIEC ZMIANY ---

# Przygotowanie danych wejściowych
X_train_dict = {'text_input': X_train_text, 'num_input': X_train_num, 'city_input': X_train_city}
X_val_dict = {'text_input': X_val_text, 'num_input': X_val_num, 'city_input': X_val_city}

y_train_dict = {'district_output': y_district_train, 'street_output': y_street_train}
y_val_dict = {'district_output': y_district_val, 'street_output': y_street_val}

# Trening
history = model.fit(
    X_train_dict, y_train_dict,
    validation_data=(X_val_dict, y_val_dict),
    epochs=20,
    batch_size=128,
    callbacks=[model_checkpoint, early_stopping]
)

Epoch 1/20
6934/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - district_output_accuracy: 0.4752 - district_output_loss: 5.2558 - loss: 13.4919 - street_output_accuracy: 0.6366 - street_output_loss: 7.1849
Epoch 1: val_loss improved from inf to 8.89180, saving model to artifacts\best_location_model.keras
6935/6935 ━━━━━━━━━━━━━━━━━━━━ 497s 71ms/step - district_output_accuracy: 0.4752 - district_output_loss: 5.2556 - loss: 13.4914 - street_output_accuracy: 0.6366 - street_output_loss: 7.1847 - val_district_output_accuracy: 0.5674 - val_district_output_loss: 3.1472 - val_loss: 8.8918 - val_street_output_accuracy: 0.6478 - val_street_output_loss: 5.1155
Epoch 2/20
6934/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - district_output_accuracy: 0.5518 - district_output_loss: 3.2085 - loss: 8.9676 - street_output_accuracy: 0.6465 - street_output_loss: 5.1174
Epoch 2: val_loss improved from 8.89180 to 7.55315, saving model to artifacts\best_location_model.keras
6935/6935 ━━━━━━━━━━━━━━━━━━━━ 523s 75ms/step 

## Przygotowanie do Predykcji Hierarchicznej
Definiujemy funkcje pomocnicze, które zaimplementują kluczową logikę biznesową: odtworzenie pełnej ścieżki lokalizacji i weryfikację jej zgodności z miastem wejściowym.

In [14]:
# KOMÓRKA 11 (Kod) - POPRAWIONA WERSJA

def get_full_path(location_id, hierarchy_map, id_to_name):
    """Iteracyjnie odtwarza pełną ścieżkę ID lokalizacji od dołu do góry."""
    path_ids = []
    current_id = location_id
    for _ in range(10): 
        if pd.notna(current_id) and current_id in id_to_name:
            path_ids.append(int(current_id))
            current_id = hierarchy_map.get(current_id)
        else:
            break
    return path_ids

# --- KLUCZOWA ZMIANA: Aktualizacja funkcji, by używała mapy odwrotnej ---
def predict_with_hierarchy(model, inputs_dict, hierarchy_map, id_to_name, inverse_target_map, top_k=5):
    """Wykonuje predykcję, odmapowuje indeksy i weryfikuje je z hierarchią."""
    # Wczytujemy mapę odwrotną, jeśli jej nie ma
    if 'inverse_target_map' not in globals():
        globals()['inverse_target_map'] = joblib.load(os.path.join(ARTIFACTS_DIR, 'inverse_target_map.joblib'))

    predictions = model.predict(inputs_dict)
    # argsort zwraca ZMAPOWANE indeksy [0, N-1]
    top_k_mapped_indices = np.argsort(predictions[0])[::-1][:top_k]
    
    input_city_id = inputs_dict['input_city'][0]
    
    for mapped_idx in top_k_mapped_indices:
        # Odmapuj indeks na oryginalne, duże ID lokalizacji
        candidate_id = inverse_target_map.get(mapped_idx)
        if candidate_id is None:
            continue # Pomiń, jeśli z jakiegoś powodu indeks jest niepoprawny
            
        path = get_full_path(candidate_id, hierarchy_map, id_to_name)
        
        if input_city_id in path:
            return candidate_id # Zwracamy ORYGINALNE ID

    # Fallback: jeśli żaden nie pasuje, zwróć pierwszego kandydata
    top_candidate_id = inverse_target_map.get(top_k_mapped_indices[0])
    return top_candidate_id
# --- KONIEC ZMIANY ---

print("Funkcje pomocnicze do predykcji hierarchicznej zdefiniowane.")

Funkcje pomocnicze do predykcji hierarchicznej zdefiniowane.


## Wykonanie i Ocena Predykcji na Zbiorze Walidacyjnym
Używamy zdefiniowanych funkcji do przeprowadzenia predykcji na zbiorze walidacyjnym i obliczamy ogólną dokładność naszego podejścia.

In [17]:
# KOMÓRKA 12 (Kod) - WERSJA BEZ PONOWNEGO TRENINGU

# --- KLUCZOWA ZMIANA: Wczytanie modelu bez kompilacji ---
# Dodajemy argument compile=False.
# To mówi Kerasowi: "Wczytaj architekturę i wagi, ale zignoruj funkcję straty i optymalizator".
# Dzięki temu unikniemy błędu związanego z niestandardową klasą HierarchicalLoss.
print(f"Wczytywanie najlepszego modelu (tylko do predykcji) z: {MODEL_PATH}")
best_model = tf.keras.models.load_model(MODEL_PATH, compile=False)
print("Model wczytany pomyślnie.")
# --- KONIEC ZMIANY ---


# Reszta kodu pozostaje bez zmian, ale dla pewności wklejam całość
# Wczytujemy mapy odwrotne dla etykiet, które stworzyliśmy w komórce 9
inv_district_map = {v: k for k, v in district_map.items()}
inv_street_map = {v: k for k, v in street_map.items()}


# Wybierzmy podzbiór danych walidacyjnych do szybkiej ewaluacji
num_samples = 1000
indices = np.random.choice(range(len(y_district_val)), num_samples, replace=False)

predicted_districts = []
predicted_streets = []
true_districts = []
true_streets = []
is_correct_list = []

print(f"Przeprowadzanie predykcji na {num_samples} próbkach...")
for i in indices:
    input_sample = {
        'text_input': X_val_text[i],
        'num_input': np.expand_dims(X_val_num[i], axis=0),
        'city_input': np.expand_dims(X_val_city[i], axis=0)
    }
    
    # Wykonujemy predykcję
    pred_district_probs, pred_street_probs = best_model.predict(input_sample, verbose=0)
    
    # Znajdujemy najbardziej prawdopodobne INDEKSY
    pred_district_idx = np.argmax(pred_district_probs[0])
    pred_street_idx = np.argmax(pred_street_probs[0])
    
    # Odmapowujemy indeksy z powrotem na ORYGINALNE ID
    predicted_district_id = inv_district_map.get(pred_district_idx, 0)
    predicted_street_id = inv_street_map.get(pred_street_idx, 0)
    
    true_district_id = inv_district_map.get(y_district_val[i], 0)
    true_street_id = inv_street_map.get(y_street_val[i], 0)
    
    # Porównujemy ORYGINALNE ID
    is_correct = (predicted_district_id == true_district_id) and \
                 ( (true_street_id == 0) or (predicted_street_id == true_street_id) )
    
    predicted_districts.append(predicted_district_id)
    predicted_streets.append(predicted_street_id)
    true_districts.append(true_district_id)
    true_streets.append(true_street_id)
    is_correct_list.append(is_correct)


# Tworzenie DataFrame z wynikami
df_results = pd.DataFrame({
    'Original_District_ID': true_districts,
    'Predicted_District_ID': predicted_districts,
    'Original_Street_ID': true_streets,
    'Predicted_Street_ID': predicted_streets,
    'Is_Correct': is_correct_list
})

# Obliczenie i wyświetlenie dokładności
accuracy = df_results['Is_Correct'].mean()
print(f"\nDokładność predykcji (Dzielnica + Ulica) na {num_samples} próbkach: {accuracy:.2%}")

# Dodawanie nazw dla czytelności
df_results['Original_District_Name'] = df_results['Original_District_ID'].apply(lambda x: id_to_name.get(x, 'Brak'))
df_results['Predicted_District_Name'] = df_results['Predicted_District_ID'].apply(lambda x: id_to_name.get(x, 'Brak'))
df_results['Original_Street_Name'] = df_results['Original_Street_ID'].apply(lambda x: id_to_name.get(x, 'Brak'))
df_results['Predicted_Street_Name'] = df_results['Predicted_Street_ID'].apply(lambda x: id_to_name.get(x, 'Brak'))

Wczytywanie najlepszego modelu (tylko do predykcji) z: artifacts\best_location_model.keras
Model wczytany pomyślnie.
Przeprowadzanie predykcji na 1000 próbkach...

Dokładność predykcji (Dzielnica + Ulica) na 1000 próbkach: 50.00%


## Wizualizacja Wyników Predykcji
Wyświetlamy losowe przykłady predykcji, kolorując wiersze na zielono (poprawne) i czerwono (błędne), aby wizualnie ocenić jakość działania modelu.

In [18]:
def highlight_correct(s):
    """Koloruje tło wiersza na podstawie wartości w kolumnie 'Is_Correct'."""
    return ['background-color: #d4edda' if s.Is_Correct else 'background-color: #f8d7da'] * len(s)

# Wyświetlenie 20 losowych wierszy ze stylizacją
styled_results = df_results.sample(min(20, len(df_results)), random_state=RANDOM_STATE).style.apply(highlight_correct, axis=1)

print("\n--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---")
display(styled_results)


--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---


,Original_District_ID,Predicted_District_ID,Original_Street_ID,Predicted_Street_ID,Is_Correct,Original_District_Name,Predicted_District_Name,Original_Street_Name,Predicted_Street_Name
521,0,0,0,0,True,Brak,Brak,Brak,Brak
737,1,1,30337,0,False,Świętokrzyskie,Świętokrzyskie,Szubin,Brak
740,2021,2021,8967,0,False,Przemyśl,Przemyśl,Zaborze,Brak
660,1302,0,0,0,False,Szczerców,Brak,Brak,Brak
411,0,0,0,0,True,Brak,Brak,Brak,Brak
678,405,0,0,0,False,Wisznice,Brak,Brak,Brak
626,0,0,0,0,True,Brak,Brak,Brak,Brak
513,0,0,0,0,True,Brak,Brak,Brak,Brak
859,0,0,27550,27550,True,Brak,Brak,Parlino,Parlino
136,146,146,6822,0,False,Wągrowiecki,Wągrowiecki,Gaj,Brak
